In [1]:
import pandas as pd

In [2]:
def dat_to_csv_converter(dat_file_path):
    # Step 1: Read the .dat file using pandas
    # Assume .dat file is comma seperated
    try:
        data = pd.read_csv(dat_file_path, delimiter=',')
        csv_file_path = dat_file_path[:-4] + ".csv"
        # Step 3: Write the data to a .csv file
        data.to_csv(csv_file_path, index=False)  # Set index=False to omit row indices in the CSV
        return csv_file_path
    except Exception as e:
        print(e)

base_file_path = '/Users/natantes/Downloads/'

final_dataframes = []

In [3]:
final_cols = [
'schoolcode',
 'year',
 'charter',
 'mathpass',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totaltested',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

def final_data_generator(csv_file_path, year):
    raw_dataframe = pd.read_csv(csv_file_path)

    working_dataframe = raw_dataframe.copy()
    working_dataframe['year'] = year
    # working_dataframe['schoolcode'] = working_dataframe['CAMPUS']
    # working_dataframe['district'] = working_dataframe['DISTRICT']
    working_dataframe['totaltested'] = working_dataframe['a1_all_d'] 
    working_dataframe['asian'] = working_dataframe['a1_etha_d']
    working_dataframe['black'] = working_dataframe['a1_ethb_d']
    working_dataframe['white'] = working_dataframe['a1_ethw_d']
    working_dataframe['hispanic'] = working_dataframe['a1_ethh_d']
    working_dataframe['lowincome'] = working_dataframe['a1_eco2_d'] + working_dataframe['a1_eco1_d']
    # working_dataframe['mathpass'] = working_dataframe['a1_all_meetsgl_nm']

    working_dataframe['asian'] /= working_dataframe['totaltested']
    working_dataframe['black'] /= working_dataframe['totaltested']
    working_dataframe['white'] /= working_dataframe['totaltested']
    working_dataframe['hispanic'] /= working_dataframe['totaltested']

    final_dataframe = working_dataframe

    print(f"Year {year} is done")

    return final_dataframe

In [4]:
# generate csv data from dat files
for year in range(2015, 2023):
    if year == 2020: continue
    current_file_path = base_file_path + "texas_math_" + str(year) + ".dat"
    final_csv_path = dat_to_csv_converter(current_file_path)
    final_dataframes.append(final_data_generator(final_csv_path, year))

final_dataframe = pd.concat(final_dataframes)
final_dataframe.head(20)

Year $2015 is done
Year $2016 is done
Year $2017 is done
Year $2018 is done
Year $2019 is done
[Errno 2] No such file or directory: '/Users/natantes/Downloads/texas_math_2020.dat'


ValueError: Invalid file path or buffer object type: <class 'NoneType'>